In [2]:
import datetime
import os
import glob
import tqdm
import numpy as np
import pandas as pd

In [3]:
flights_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/preprocess/flights.csv"
default_spire_path = "/home/jo-kwsm/synology/horie/nict/data/spire/*"
time_data_path = "../../data/OAG.csv"
airport_path = "data/airports.csv"

In [3]:
time_data = pd.read_csv(time_data_path)

/home/jo-kwsm/.pyenv/versions/3.8.5/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
flights_sample_path = default_flights_path %("01", "0101")
flights_sample = pd.read_csv(flights_path)
oag_sample_path = "../../data/20190101_flights.csv"
oag_sample = pd.read_csv(oag_sample_path)

In [8]:
oag_sample.iloc[0]

flight_id      520668873
aircraft_id      7459875
reg                  NaN
equip                NaN
callsign             NaN
flight               NaN
schd_from            ICN
schd_to              NaN
real_to              NaN
reserved             NaN
Name: 0, dtype: object

In [26]:
print(len(data))

1777


In [11]:
flights_data = pd.DataFrame()
for month in range(1, 13):
    for date in range(1,32):
        month_str = str(month)
        if len(month_str) == 1:
            month_str = "0" + month_str
        date_str = str(date)
        if len(date_str) == 1:
            date_str = "0" + date_str
        date_str = month_str+date_str
        flights_path = os.path.join(default_flights_path%(month_str, date_str))

        if not os.path.exists(flights_path):
            continue

        date_data = pd.read_csv(flights_path)
        # date_data = date_data[(date_data["schd_from"] == "HND") | (date_data["schd_from"] == "OKA")]
        # date_data = date_data[(date_data["schd_to"] == "HND") | (date_data["schd_to"] == "OKA")]
        date_data = date_data[date_data["schd_from"] == "HND"]
        date_data = date_data[date_data["schd_to"] == "OKA"]
        # print((date_data["schd_from"] == "OKA").sum())
        flights_data = flights_data.append(date_data)
        if month == 0 and date == 0:
            print(len(date_data["flight_id"]))
            print(date_data["equip"].unique())
            print(len(date_data[date_data["equip"] == "B773"]))
        # for flight_id in date_data["flight_id"]:
        #     print(flight_id)

In [4]:
flights_data = pd.read_csv(flights_path)
print(len(flights_data))

11014


In [13]:
flights_data.to_csv("../../data/flights.csv")

In [5]:
spire_data = pd.DataFrame()
for i, path in tqdm.tqdm(enumerate(glob.glob(default_spire_path))):
    spire_path = path
    data = pd.read_csv(spire_path, low_memory=False)
    # data = data[(data["origin_airport_icao"] == "ROAH") | (data["origin_airport_icao"] == "RJTT")]
    # data = data[(data["destination_airport_icao"] == "ROAH") | (data["destination_airport_icao"] == "RJTT")]
    data = data[data["origin_airport_icao"] == "RJTT"]
    data = data[data["destination_airport_icao"] == "ROAH"]
    spire_data = spire_data.append(data)
    # if i < 0:
    #     data = data[data["timestamp"] < "2019-01-02"]
    #     print(len(data))
    #     print(data["icao_actype"].unique())
    #     # print(data["scheduled_departure_time_utc"].unique())
    #     # print(data["scheduled_arrival_time_utc"].unique())
    #     print(len(data[data["icao_actype"] == "B773"]["scheduled_departure_time_utc"].unique()))
    #     print(len(data[data["icao_actype"] == "B773"]["scheduled_arrival_time_utc"].unique()))

546it [53:07,  5.84s/it]


In [6]:
spire_data.drop_duplicates(subset=["icao_actype", "scheduled_departure_time_utc"], inplace=True)
spire_data.dropna(subset=["scheduled_departure_time_utc", "scheduled_arrival_time_utc"], inplace=True)
sched_departure_timestamp = []
sched_arrival_timestamp = []
for i in range(len(spire_data)):
    try:
        sched_departure_timestamp.append(datetime.datetime.strptime(spire_data.iloc[i]["scheduled_departure_time_utc"].replace("Z", ""), "%Y-%m-%dT%H:%M:%S").timestamp())
        sched_arrival_timestamp.append(datetime.datetime.strptime(spire_data.iloc[i]["scheduled_arrival_time_utc"].replace("Z", ""), "%Y-%m-%dT%H:%M:%S").timestamp())
    except:
        sched_departure_timestamp.append(spire_data.iloc[i]["scheduled_departure_time_utc"])
        sched_arrival_timestamp.append(spire_data.iloc[i]["scheduled_arrival_time_utc"])
        print(i, spire_data.iloc[i]["scheduled_departure_time_utc"], spire_data.iloc[i]["scheduled_arrival_time_utc"])
spire_data["scheduled_departure_timestamp"] = pd.Series(sched_departure_timestamp)
spire_data["scheduled_arrival_timestamp"] = pd.Series(sched_arrival_timestamp)

In [51]:
print(len(spire_data))
print(len(flights_data))

6570
11014


In [57]:
flight = flights_data[flights_data["flight_id"] == 593293511]
print(len(flight))

1


In [5]:
try:
    spire_data.to_csv("../../data/spire.csv")
    print("save spire.")
except:
    spire_data = pd.read_csv("../../data/spire.csv")
    print("read spire.")

read spire.


In [6]:
def spire_and_flightradar(candidate):
    candidate_list = []
    default_position_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/positions"
    for flight_csv_path in tqdm.tqdm(candidate):
        flight_csv_path = flight_csv_path[:-1]
        flight_id = int(flight_csv_path.split(".")[0])
        position_path = os.path.join(default_position_path, flight_csv_path)
        positions = pd.read_csv(position_path)
        dep_timestamp = positions.iloc[0]["snapshot_id"]
        arr_timestamp = positions.iloc[-1]["snapshot_id"]
        flight = flights_data[flights_data["flight_id"] == flight_id]

        if len(flight) < 1:
            print("There isn't %s in flight data." % flight_id, type(flight_id))
            continue

        icao_code = flight.iloc[0]["equip"]
        timetables = spire_data[spire_data["icao_actype"] == icao_code]

        for i in range(len(timetables)):
            timetable = timetables.iloc[i]
            sch_from = timetable["scheduled_departure_timestamp"]
            sch_to = timetable["scheduled_arrival_timestamp"]

            if sch_from - 3600 * 2 < dep_timestamp and arr_timestamp < sch_to + 3600 * 2:
                candidate_list.append(flight_csv_path)
                break
    return candidate_list

In [7]:
print(type(flights_data.iloc[0]["flight_id"]))
print(flights_data.head())

<class 'numpy.int64'>
   flight_id  aircraft_id     reg equip callsign flight schd_from schd_to  \
0  520767396      8816572  JA75AN  B738   ANA461  NH461       HND     OKA   
1  520768036      8723556  JA345J  B738   JAL991  JL991       HND     OKA   
2  520769002      8814651  JA737R   738   SKY511  BC511       HND     OKA   
3  520770574      8811922  JA708A  B772   ANA993  NH993       HND     OKA   
4  520771366      8651003  JA007D  B772      NaN  JL901       HND     OKA   

  real_to  reserved  departure_day  Arrival_day  
0     OKA       NaN       20190101     20190101  
1     OKA       NaN       20190101     20190101  
2     OKA       NaN       20190101     20190101  
3     OKA       NaN       20190101     20190101  
4     OKA       NaN       20190101     20190101  


In [8]:
test_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/test.txt"
train_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/train.txt"
val_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/val.txt"
data_candidate = []
with open(test_path, "r") as f:
    candidate = f.readlines()
    for c in candidate:
        data_candidate.append(c)
with open(train_path, "r") as f:
    candidate = f.readlines()
    for c in candidate:
        data_candidate.append(c)
with open(val_path, "r") as f:
    candidate = f.readlines()
    for c in candidate:
        data_candidate.append(c)

test_candidate_list = spire_and_flightradar(data_candidate)

100%|██████████| 8876/8876 [23:47<00:00,  6.22it/s]


In [9]:
print(len(data_candidate))
print(len(test_candidate_list))

8876
2782


In [16]:
new_test_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/new_test.txt"
new_train_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/new_train.txt"
new_val_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/new_val.txt"
with open(new_test_path, "w") as f:
    f.write("\n".join(test_candidate_list[:1000]))
    f.write("\n")
with open(new_val_path, "w") as f:
    f.write("\n".join(test_candidate_list[1000:2000]))
    f.write("\n")
with open(new_train_path, "w") as f:
    f.write("\n".join(test_candidate_list[2000:]))
    f.write("\n")
    for d in data_candidate:
        if not d in test_candidate_list:
            f.write(d)

In [14]:
new_test_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/new_test.txt"
new_train_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/new_train.txt"
new_val_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/new_val.txt"

with open(new_test_path, "r") as f:
    with open(new_test2_path, "r") as nf:
        c = f.readlines()
        nc = nf.readlines()
        if not len(c) == len(nc):
            print("test out")
        for i in tqdm.tqdm(range(len(c))):
            if not c[i] == nc[i]:
                print("test", i, "out")
with open(new_train_path, "r") as f:
    with open(new_train2_path, "r") as nf:
        c = f.readlines()
        nc = nf.readlines()
        print(len(c))
        print(len(nc))
        if not len(c) == len(nc):
            print("train out")
        for i in tqdm.tqdm(range(len(nc))):
            if not c[i] == nc[i] and i < 800:
                print("train", i, "out")
with open(new_val_path, "r") as f:
    with open(new_val2_path, "r") as nf:
        c = f.readlines()
        nc = nf.readlines()
        if not len(c) == len(nc):
            print("val out")
        for i in tqdm.tqdm(range(len(c))):
            if not c[i] == nc[i]:
                print("val", i, "out")

100%|██████████| 1000/1000 [00:00<00:00, 2924898.19it/s]18534
9658
train out
train 783 out
train 784 out
train 785 out
train 786 out
train 787 out
train 788 out
train 789 out
train 790 out
train 791 out
train 792 out
train 793 out
train 794 out
train 795 out
train 796 out
train 797 out
train 798 out
train 799 out



In [12]:
airport_path = "../../data/airports.csv"
airport_info = pd.read_csv(airport_path)
oka = airport_info[airport_info["iata_code"] == "OKA"]
hnd = airport_info[airport_info["iata_code"] == "HND"]
airport2point = {
    "OKA": (oka.iloc[0]["latitude_deg"], oka.iloc[0]["longitude_deg"]),
    "HND": (hnd.iloc[0]["latitude_deg"], hnd.iloc[0]["longitude_deg"]),
}

In [20]:
def divide(dep_point, arr_point, dep_datetime, arr_datetime, judge_datetime):
        """
        内分計算によって座標を求める。

        Args:
            dep_point(float):出発地点(deg)
            arr_point(float):到着地点(deg)
            dep_datetime(datetime):出発時刻
            arr_datetime(datetime):到着時刻
            judge_datetime(datetime):位置を知りたい時刻
        Returns:
            point(float):judge_datetimeにおける地点
        """

        point = ((arr_datetime - judge_datetime) * dep_point \
                + (judge_datetime - dep_datetime) * arr_point) \
                / (arr_datetime - dep_datetime)
        # print(
        #     (arr_datetime - judge_datetime),
        #     (judge_datetime - dep_datetime),
        #     (arr_datetime - dep_datetime),
        # )
        return point

def estimate_position(sch_from, sch_to, judge_datetime):
    dep_airport = "HND"
    arr_airport = "OKA"
    dep_point = airport2point[dep_airport]
    arr_point = airport2point[arr_airport]
    # if arr_airport != "OKA":
    #     print(arr_airport)
    point = (
        divide(
            dep_point[0],
            arr_point[0],
            sch_from,
            sch_to,
            judge_datetime,
        ),
        divide(
            dep_point[1],
            arr_point[1],
            sch_from,
            sch_to,
            judge_datetime,
        ),
    )
    return point

In [23]:
default_position_path = "/home/jo-kwsm/synology/horie/nict/data/flightradar24_processed/20190101-20191231_HND-OKA/spline/positions"
result = {
    "snapshot_id": [],
    "flight_id": [],
    "Predicted_lat": [],
    "Predicted_lon": [],
}
with open(new_test_path, "r") as f:
    test_data = f.readlines()
    for flight_csv_path in tqdm.tqdm(test_data):
        flight_csv_path = flight_csv_path[:-1]
        flight_id = int(flight_csv_path.split(".")[0])
        test_position_path = os.path.join(default_position_path, flight_csv_path)
        test_positions = pd.read_csv(test_position_path)
        dep_timestamp = test_positions.iloc[0]["snapshot_id"]
        arr_timestamp = test_positions.iloc[-1]["snapshot_id"]
        flight = flights_data[flights_data["flight_id"] == flight_id]

        if len(flight) < 1:
            print("There isn't %s in flight data." % flight_id, type(flight_id))
            continue

        icao_code = flight.iloc[0]["equip"]
        timetables = spire_data[spire_data["icao_actype"] == icao_code]

        for i in range(len(timetables)):
            timetable = timetables.iloc[i]
            sch_from = timetable["scheduled_departure_timestamp"]
            sch_to = timetable["scheduled_arrival_timestamp"]

            if sch_from - 3600 * 2 < dep_timestamp and arr_timestamp < sch_to + 3600 * 2:
                break

        for i in range(len(test_positions)):
            test_position = test_positions.iloc[i]
            test_timestamp = test_position["snapshot_id"]
            if not test_timestamp % (60*5) == 0:
                continue
            lat = np.nan
            lon = np.nan
            if sch_from < test_timestamp and test_timestamp < sch_to:
                lat, lon = estimate_position(sch_from, sch_to, test_timestamp)
            if test_timestamp <= sch_from:
                lat = airport2point["HND"][0]
                lon = airport2point["HND"][1]
            if  sch_to <= test_timestamp:
                lat = airport2point["OKA"][0]
                lon = airport2point["OKA"][1]
            result["snapshot_id"].append(test_position["snapshot_id"])
            result["flight_id"].append(flight_id)
            result["Predicted_lat"].append(lat)
            result["Predicted_lon"].append(lon)
            


100%|██████████| 1000/1000 [02:19<00:00,  7.19it/s]


In [105]:
spire_data.

,latitude,longitude,altitude_baro,speed,heading,vertical_rate,squawk_code
count,436259.000000,436259.000000,414111.000000,414695.000000,435616.000000,374451.000000,162349.000000
mean,31.394873,133.914153,22610.899010,308.565917,222.064238,18.057551,3051.407419
std,3.909396,4.930762,12429.259803,105.698614,49.379570,1359.557707,1727.099982
min,25.516296,126.921563,-450.000000,0.000000,0.000000,-6590.000000,401.000000
25%,26.926466,128.117118,11050.000000,260.000000,210.000000,-700.000000,2072.000000
50%,33.613509,136.329746,26000.000000,340.000000,220.000000,0.000000,2376.000000
75%,35.007888,138.353435,32875.000000,380.000000,250.000000,60.000000,4601.000000
max,35.742203,140.318356,63800.000000,600.000000,360.000000,8000.000000,7377.000000


In [24]:
result_path = "../../data/timetable_result.csv"
result_df = pd.DataFrame(result, columns=["snapshot_id", "flight_id", "Predicted_lat", "Predicted_lon"])
result_df.to_csv(result_path, index=None)

In [ ]:
# spireに存在してflightradarに存在しないものの調査
# 発着時刻より遅い場合、早い場合は空港を出力とした。